In [ ]:
!pip install -q -U transformers==4.37.2
!pip install peft
!pip install -q bitsandbytes==0.41.3 accelerate==0.25.0

In [ ]:
import torch
print(torch.__version__)

2.2.1+cu121


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
from PIL import Image
import os
import numpy as np
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
from transformers import AutoProcessor, BlipForQuestionAnswering, LlavaForConditionalGeneration, Pix2StructForConditionalGeneration
from transformers import BitsAndBytesConfig

model_select = "llava" # @param ["blip", "llava", "bakllava", "matcha-p", "matcha-c"]
model_id = None
model = None

if model_select == "llava":
  model_id = "llava-hf/llava-1.5-7b-hf"
  model = LlavaForConditionalGeneration.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")
elif model_select == "bakllava":
  model_id = "llava-hf/bakLlava-v1-hf"
  model = LlavaForConditionalGeneration.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")
elif model_select == "matcha-p":
  model_id = "google/matcha-plotqa-v1"
  model = Pix2StructForConditionalGeneration.from_pretrained(model_id)
elif model_select == "matcha-c":
  model_id = "google/google/matcha-chartqa"
  model = Pix2StructForConditionalGeneration.from_pretrained(model_id)
elif model_select == "blip":
  model_id = "Salesforce/blip-vqa-base"
  model = BlipForQuestionAnswering.from_pretrained(model_id)

processor = AutoProcessor.from_pretrained(model_id)

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [ ]:
print(model)

In [ ]:
!ls drive/MyDrive/SMART/data/

101  66  71  75  95  99			   test_data_bakllava_hp2.json	test_data_mpqa_p.json
61   67  72  76  96  icon-classes.txt	   test_data_bakllava_hp.json
62   69  73  77  97  puzzle_14_e_1.png	   test_data_bakllava.json
65   70  74  94  98  puzzle_type_info.csv  test_data_llava_rem.json


In [ ]:
DATASET_DIR = "drive/MyDrive/SMART/data/"

In [ ]:
def read_csv(csvfilename, puzzle_id):
    import csv
    qa_info = []
    with open(csvfilename, newline="") as csvfile:
        datareader = csv.DictReader(csvfile)
        for row in datareader:
            row["puzzle_id"] = str(puzzle_id)
            if len(row["A"]) == 0:
                row["A"] = "A"
                row["B"] = "B"
                row["C"] = "C"
                row["D"] = "D"
                row["E"] = "E"
            qa_info.append(row)
    return qa_info

SEQ_PUZZLES = [16, 18, 35, 39, 63, 100]
SIGNS = np.array(["+", "-", "x", "/"])
MAX_DECODE_STEPS = 10

def get_puzzle_class_info(puzzle_ids, icon_class_ids):
    #    global SEQ_PUZZLES, puzzle_diff_str, puzzle_diff
    puzzle_classes = {}
    for puzzle_id in puzzle_ids:
        puzzle_root = puzzle_id
        csv_file = "puzzle_%s.csv" % (puzzle_id)
        qa_info = read_csv(os.path.join(DATASET_DIR, puzzle_root, csv_file), puzzle_id)

        pid = int(puzzle_id)
        if pid not in SEQ_PUZZLES:
            num_classes = np.array([get_val(qa, qa["Answer"], {}, icon_class_ids) for qa in qa_info]).max() + 1
        else:
            if pid in [16, 39, 100]:
                num_classes = 26 + 1  # if the output is a string of numbers, and the max classes is - max val.
            elif pid in [18, 35]:
                num_classes = 5 + 1  # the minus one is for end of items.
            elif pid in [63]:
                num_classes = np.array([get_val(qa, qa["Answer"], {}, icon_class_ids).max() for qa in qa_info]).max() + 1
        puzzle_classes[str(puzzle_id)] = num_classes
    return puzzle_classes

def get_icon_dataset_classes(icon_path):
    """returns the classes in ICONs-50 dataset"""
    with open(icon_path, "r") as f:
        icon_classes = f.readlines()
    return [ii.rstrip() for ii in icon_classes]

def str_replace(ans):
    ans = ans.replace(" hours", "")
    ans = ans.replace(" hour", "").replace(" cm", "")
    ans = ans.replace(" km", "")
    return ans

def pad_with_max_val(gt_list, val):
    """if the number of elements in gt is less than MAX_DECODE_STEPS, we pad it with the max value in a class"""
    if len(gt_list) < MAX_DECODE_STEPS:
        gt_list = (
            gt_list
            + (
                np.ones(
                    MAX_DECODE_STEPS - len(gt_list),
                )
                * val
            ).tolist()
        )
    return gt_list

def get_val(qinfo, ans_opt, num_classes_per_puzzle, icon_class_ids, is_one_of_option=False):
    """get the value of the answer option. This code also encodes the value into a number by removing extreneous strings"""
    """ is_one_of_option is True, when ans_opt is one of the options, need not be the correct answer option."""
    where = lambda x, y: np.where(np.array(x) == y)[0][0]
    pid = int(qinfo["puzzle_id"])
    if pid in SEQ_PUZZLES:
        ans = qinfo[ans_opt]
        if pid == 16:
            ans_opt_val = [int(ii) for ii in ans.replace("and", ",").replace(", ,", ",").replace(" ", "").split(",")]
            ans_opt_val = pad_with_max_val(ans_opt_val, 26)
        elif pid == 18:
            ans_opt_val = [int(ii) for ii in ans.split("-")]
            ans_opt_val = pad_with_max_val(ans_opt_val, 5)
        elif pid == 35:
            ans_opt_val = [
                ord(ii) - ord("A") for ii in ans.replace("and", ",").replace(", ,", ",").replace(" ", "").split(",")
            ]
            ans_opt_val = pad_with_max_val(ans_opt_val, 5)
        elif pid == 39:
            ans_opt_val = [ord(ii) - ord("A") for ii in list(ans)]
            ans_opt_val = pad_with_max_val(ans_opt_val, 26)
        elif pid == 63:
            ans_opt_val = [
                int(ii)
                for ii in ans.replace("and", ",")
                .replace("or", ",")
                .replace(", ,", ",")
                .replace("only", "")
                .replace(" ", "")
                .split(",")
            ]
            key = str(63)
            if key in num_classes_per_puzzle:
                ans_opt_val = pad_with_max_val(ans_opt_val, num_classes_per_puzzle[key] - 1)
        elif pid == 100:
            ans_opt_val = [ord(ii) - ord("A") for ii in list(ans)]
            ans_opt_val = pad_with_max_val(ans_opt_val, 26)
        ans_opt_val = np.array(ans_opt_val)

    elif pid == 58:
        # puzzle 58 has answers as <operator><one digit number>, e.g./4,-5, etc.
        # we use +=1, -=2, x=3, /=4. so /4 will be 44, -5=25, +2= 2.
        ans_opt_val = qinfo[ans_opt]
        ans_opt_val = (where(SIGNS, ans_opt_val[0]) + 1) * 10 + int(ans_opt_val[1:])
    elif pid == 25:
        # we need to fix the time in AM/PM format properly.
        ans = qinfo[ans_opt]
        ans_opt_val = int(ans.replace(":00 AM", "").replace(":00 PM", ""))
        if ans.find("PM") > -1:
            ans_opt_val += 12
    else:
        try:
            ans_opt_val = int(qinfo[ans_opt])
        except:
            if len(qinfo[ans_opt]) > 0:
                try:
                    ans_opt_val = ord(qinfo[ans_opt]) - ord("A")
                except:
                    try:
                        ans_opt_val = str_replace(qinfo[ans_opt])
                        ans_opt_val = ans_opt_val.replace("Impossible", "0")  # puzzle 58.
                        if int(qinfo["puzzle_id"]) == 1:  # if the puzzle id is 1, then the options are icon classes.
                            ans_opt_val = "_".join(ans_opt_val.split(" "))
                            if ans_opt_val in icon_class_ids:
                                ans_opt_val = where(icon_class_ids, ans_opt_val)
                            elif ans_opt_val + "s" in icon_class_ids:
                                ans_opt_val = where(icon_class_ids, ans_opt_val + "s")
                        ans_opt_val = int(ans_opt_val)
                    except:
                        print(qinfo)
                        pdb.set_trace()
            else:
                ans_opt_val = ord(ans_opt) - ord("A")
    if not is_one_of_option:  # implies we are encoding the correct answer.
        qinfo["AnswerValue"] = ans_opt_val
    return ans_opt_val

In [ ]:
def split_data(info, split):
    """
    split_type=standard is to use the split_ratio in the instance order
    split_type=exclude is to exclude answers from the split, e.g., train on all answers except say 1, and test 1
    split_type=puzzle is to split the puzzles into the respective ratios. so we don't have to do anything here.
    """
    split_ratio = "80:5:15"
    splits = np.array([int(spl) for spl in split_ratio.split(":")]).cumsum()
    n = len(info)
    if split == "train":
        st = 0
        en = int(np.floor(n * splits[0] / 100.0))
        info = info[st:en]
    elif split == "val":
        st = int(np.ceil(n * splits[0] / 100.0))
        en = int(np.floor(n * splits[1] / 100.0))
        info = info[st:en]
    else:
        st = int(np.ceil(n * splits[1] / 100.0))
        info = info[st:]
    return info

In [ ]:
import random
random.seed(1007)

PS_VAL_IDX = [7, 43, 64]
PS_TEST_IDX = [94, 95, 96, 97, 98, 99, 101, 61, 62, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77]

def train_data():
  MAX_VAL = 0
  qa_info = []
  icon_class_ids = get_icon_dataset_classes(DATASET_DIR + "icon-classes.txt")

  puzzle_ids = os.listdir(DATASET_DIR)
  puzzle_ids = np.array(puzzle_ids)[np.array([x.find(".") == -1 for x in puzzle_ids])]
  puzzle_ids = puzzle_ids.tolist()
  val_test = PS_VAL_IDX + PS_TEST_IDX
  val_test = set([str(ii) for ii in val_test])
  puzzle_ids = list(set(puzzle_ids).difference(val_test))

  num_classes_per_puzzle = get_puzzle_class_info(puzzle_ids, icon_class_ids)
  print("number of train puzzles = %d" % (len(puzzle_ids)))
  for puzzle_id in puzzle_ids:
      csv_file = "puzzle_%s.csv" % (puzzle_id)
      tqa_info = read_csv(os.path.join(DATASET_DIR, puzzle_id, csv_file), puzzle_id)
      for t in range(len(tqa_info)):
          tqa_info[t]["AnswerValue"] = get_val(tqa_info[t], tqa_info[t]["Answer"], num_classes_per_puzzle, icon_class_ids)
      qa_info = qa_info + split_data(tqa_info, "train")
  print(len(qa_info))
  return qa_info

def test_data(split):
  MAX_VAL = 0
  qa_info = []
  icon_class_ids = get_icon_dataset_classes(DATASET_DIR + "icon-classes.txt")
  puzzle_ids = []
  if split == "test":
    puzzle_ids = [str(ii) for ii in PS_TEST_IDX]
  else:
    puzzle_ids = [str(ii) for ii in PS_VAL_IDX]
  num_classes_per_puzzle = get_puzzle_class_info(puzzle_ids, icon_class_ids)
  print("number of %s puzzles = %d" % (split, len(puzzle_ids)))
  for puzzle_id in puzzle_ids:
      csv_file = "puzzle_%s.csv" % (puzzle_id)
      tqa_info = read_csv(os.path.join(DATASET_DIR, puzzle_id, csv_file), puzzle_id)
      for t in range(len(tqa_info)):
          tqa_info[t]["AnswerValue"] = get_val(tqa_info[t], tqa_info[t]["Answer"], num_classes_per_puzzle, icon_class_ids)
      qa_info = qa_info + split_data(tqa_info, split)
  print(len(qa_info))
  random.shuffle(qa_info)
  return qa_info

# train_qa_info = train_data()
# val_qa_info = test_data("val")
test_qa_info = test_data("test")

number of test puzzles = 21
6300


In [ ]:
import json
import matplotlib.pyplot as plt
from tqdm import tqdm

puzzle_ids = os.listdir(DATASET_DIR)
puzzle_ids = np.array(puzzle_ids)[np.array([x.find(".") == -1 for x in puzzle_ids])]
puzzle_ids = puzzle_ids.tolist()
icon_class_ids = get_icon_dataset_classes(DATASET_DIR + "icon-classes.txt")
num_classes_per_puzzle = get_puzzle_class_info(puzzle_ids, icon_class_ids)

def str_replace_(info, ans_opt):
    ans = info[ans_opt]
    ans = ans.replace(" hours", "")
    ans = ans.replace(" hour", "").replace(" cm", "")
    ans = ans.replace(" km", "")
    ans = ans.replace("Impossible", "0")
    info[ans_opt] = ans
    return ans

def getitem(qa_info, idx):
    info = qa_info[idx]
    pid = info["puzzle_id"]
    puzzle_root = info["puzzle_id"] + "/"
    im = Image.open(os.path.join(DATASET_DIR, puzzle_root, "img", info["image"]))
    qa = info["Question"]
    _ = [str_replace_(info, key) for key in ["A", "B", "C", "D", "E"]]
    opts = [get_val(info, key, num_classes_per_puzzle, icon_class_ids, is_one_of_option=True) for key in ["A", "B", "C", "D", "E"]]
    lbl = info["Answer"]
    answer_value = info["AnswerValue"]
    answer = np.zeros(MAX_DECODE_STEPS,)
    if int(pid) not in SEQ_PUZZLES:
        answer[0] = answer_value
    else:
        answer[: len(answer_value)] = answer_value
    return info["image"], im, qa, opts, lbl, answer_value, info["puzzle_id"]

In [ ]:
from collections import defaultdict
import csv

def prepare_test_batch(qa_info, i, batch_size, len_data):
  test_dict = defaultdict(dict)
  imgs = []
  prompts = []
  device = "cuda" if torch.cuda.is_available() else "cpu"
  opts = ["A", "B", "C", "D", "E"]
  puzzle_info = {}
  with open(DATASET_DIR + "puzzle_type_info.csv", 'r') as openfile:
      reader = csv.DictReader(openfile)
      for row in reader:
          puzzle_id = row['puzzle_id']
          puzzle_type = row['type']
          puzzle_info[puzzle_id] = puzzle_type
  for j in range(i, min(i+batch_size, len_data)):
    test_dict[j]["im"], im, test_dict[j]["qa"], test_dict[j]["opts"], test_dict[j]["lbl"], test_dict[j]["answer_value"], test_dict[j]["puzzle_id"] = getitem(qa_info, j)
    if model_id == "llava" or model_id == "bakllava":
      input = "<s> USER: <image>\n {} \nOptions: {} \nASSISTANT: ".format(test_dict[j]["qa"], " ".join(["({}) {}".format(opt, str(val)) for opt, val in zip(opts, test_dict[j]["opts"])]))
    elif model_id == "matcha-p" or model_id == "matcha-c":
      input = "Answer with the option’s letter from the given choices directly. Q: {} Options: {}".format(test_dict[j]["qa"], " ".join(["({}) {}".format(opt, str(val)) for opt, val in zip(opts, test_dict[j]["opts"])]))
    elif model_id == "blip":
      input = test_dict[j]["qa"]
    test_dict[j]["input"] = input
    test_dict[j]["type"] = puzzle_info[test_dict[j]["puzzle_id"]]
    imgs.append(im)
    prompts.append(test_dict[j]["input"])
  inputs = processor(images=imgs, text=prompts, padding=True, return_tensors="pt").to(device)
  return test_dict, inputs

def test_extreme_generalization(model, qa_info, batch_size=16):
  print("Batch size: ", batch_size)
  accuracy = 0.0
  num_correct = 0
  num_total = 0
  len_data = len(qa_info)
  with open(DATASET_DIR + "test_data.json", "w+") as fh:
      print("[", file=fh)
      for i in tqdm(range(0, len_data, batch_size)):
          test_dict, inputs = prepare_test_batch(qa_info, i, batch_size, len_data)
          with torch.no_grad():
            predictions = model.generate(**inputs, max_new_tokens=512)
          answers = processor.batch_decode(predictions, skip_special_tokens=True)
          for j in range(len(answers)):
            test_dict[i+j]["answer"] = answers[j]
            if test_dict[i+j]["answer"] == str(test_dict[i+j]["answer_value"]):
                num_correct += 1
            json.dump(test_dict[i+j], fh)
            print(",", file=fh)
          num_total += batch_size
          accuracy = num_correct / num_total
          print("Accuracy: " + format(accuracy, '.2f'))
      print("]", file=fh)


In [ ]:
# Zero-shot extreme generalization
model.to("cuda")
test_extreme_generalization(model, test_qa_info)

Batch size:  16


  0%|          | 1/394 [00:01<09:46,  1.49s/it]

Accuracy: 0.00


  1%|          | 2/394 [00:12<45:29,  6.96s/it]

Accuracy: 0.00


...

Accuracy: 0.04


100%|█████████▉| 393/394 [47:01<00:08,  8.32s/it]

Accuracy: 0.04


100%|██████████| 394/394 [47:06<00:00,  7.17s/it]

Accuracy: 0.04


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
account = "akankshya107"
peft_model_id = f"{account}/bakllava-base"
model.push_to_hub(peft_model_id)

Removed shared tensor {'word_embeddings.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading


KeyboardInterrupt: 